In [ ]:
from openai import OpenAI
import json
import os


In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY']
)

In [ ]:
client = OpenAI()

## Company generation

In [ ]:
system_prompt = (
    "You are an expert in the Italian economy, business landscape, and regional industries. "
    "Your expertise allows you to generate realistic company profiles that reflect authentic market "
    "conditions, financial metrics, and international trade connections."
)
user_prompt = (
    "Generate 1 fictional companies based in Italy, reflecting a realistic distribution in regional "
    "diversity, income and sector. Each company should reflect Italy’s economic landscape, cultural heritage, "
    "and business environment while maintaining realism in its size, revenue, and international reach.\n\n"
    "For each company, structure the response in the exact order below, using only the values (no labels and categories):\n"
    "- Business name\n- Year of founding\n- Core products/services\n- Additional background\n"
    "- City and region\n- Employee count range\n- Annual revenue range\n"
    "- Primary client base\n- National or international scope"
)

functions = [
    {
        "name": "get_company_info",
        "description": "Generate structured profiles for fictional companies in Italy",
        "parameters": {
            "type": "object",
            "properties": {
                "company": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                          "company_name":              { "type": "string" },
                          "establishment_year":        { "type": "string" },
                          "offered_products_services": { "type": "string" },
                          "company_details":           { "type": "string" },
                          "headquarters_location":     { "type": "string" },
                          "number_of_employees":       { "type": "string" },
                          "annual_revenue":            { "type": "string" },
                          "main_consumer":             { "type": "string" },
                          "affairs_extent":            { "type": "string" }
                        },
                        "required": [
                            "company_name",
                            "establishment_year",
                            "offered_products_services",
                            "company_details",
                            "headquarters_location",
                            "number_of_employees",
                            "annual_revenue",
                            "main_consumer",
                            "affairs_extent"
                        ],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["company"],
            "additionalProperties": False
        }
    }
]

response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ],
    functions=functions,
    function_call={"name": "get_company_info"},
    temperature=1
)


In [ ]:
company = json.loads(response.choices[0].message.function_call.arguments)['company'][0]

for key, value in company.items():
    print(f"{key}: {value}")


company_name: VinoVeritas S.p.A.
establishment_year: 1978
offered_products_services: Premium Italian wines and vineyard tours
company_details: VinoVeritas S.p.A. is renowned for producing premium wines with a focus on traditional Italian wine-making techniques. Nestled in the rolling hills of Tuscany, the company operates one of the most scenic vineyards in the region. They offer guided tours, wine tasting experiences, and cultural events that attract tourists from all over the world, showcasing the beauty and heritage of Italian wine.
headquarters_location: Florence, Tuscany
number_of_employees: 150-200
annual_revenue: €25-40 million
main_consumer: Wine enthusiasts and international tourists
affairs_extent: International


## Employees

In [ ]:
system_prompt = (
  "You are an expert in characterizing realistic people based on the companies they work for."
)
company_str = json.dumps(company, ensure_ascii=False, indent=2)
user_prompt = f"""
  Generate 1 realistic employee profiles for a company, specifically create profiles of those who are likely to use personal computers in their daily work.
  Ensure to generate a mixed but realistic set of employees that covers senior, mid-level, and junior roles.

  Company profile:{company_str}

  Each profile must include the following characteristics:
  - Realistic and original name;
  - Gender (M for male / F for female);
  - Realistic age for the role;
  - Birthplace (City, Country);
  - Relevant qualifications and institutions;
  - Known languages and proficiency levels;
  - Job title (aligned with the company’s sector and organizational structure);
  - Current project descriptions tied to the company's services and employee role;
  - Time employed (e.g., “X years”, aligned with role seniority);
  - Tech proficiency (Basic / Intermediate / Advanced / Expert);
  - Relevant hobbies and free-time activities;
  - Social media platforms;
"""




employee_schema = {
    "name": "get_employee_info",
    "description": "Generate structured profiles for fictional employees working in the previously generated company",
    "parameters": {
    "type": "object",
    "properties": {
        "employee_profiles": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name":            {"type": "string"},
                    "gender":          {"type": "string"},
                    "age":             {"type": "number"},
                    "birthplace":      {"type": "string"},
                    "qualifications":  {"type": "string"},
                    "languages":       {"type": "string"},
                    "job_title":       {"type": "string"},
                    "current_project": {"type": "string"},
                    "time_employed":   {"type": "string"},
                    "tech_proficiency":{"type": "string"},
                    "hobbies":         {"type": "string"},
                    "social_media":    {"type": "string"}
                },
                "required": [
                    "name","gender","age","birthplace","qualifications",
                    "languages","job_title","current_project","time_employed",
                    "tech_proficiency","hobbies","social_media"
                ],
                "additionalProperties": False
            }
        }
    },
    "required": ["employee_profiles"],
    "additionalProperties": False
    }
}

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ],
    functions=[employee_schema],
    function_call={"name": "get_employee_info"},
    temperature=1.2
)


In [ ]:
employee = json.loads(response.choices[0].message.function_call.arguments)['employee_profiles'][0]

for key, value in employee.items():
    print(f"{key}: {value}")

name: Giulia Ferraro
gender: F
age: 45
birthplace: Florence, Italy
qualifications: Master’s in Enology from the University of Florence
languages: Italian (Native), English (Fluent), French (Intermediate)
job_title: Senior Wine Production Manager
current_project: Overseeing the production of the new 'Reserve Collection' lines and implementing sustainable practices in vineyard management.
time_employed: 20 years
tech_proficiency: Advanced
hobbies: Wine tasting, painting, hiking
social_media: LinkedIn, Instagram


## Ham Email Scenario Creation

In [ ]:
system_prompt = """
You are an AI an expert in email communication and in writing email characteristics based on employee's profile and company details.
"""

user_prompt = f"""
Create 1 appropriate characteristics for plausible emails that the employee might receive.

The characteristics should focus on internal and external communications, based on the company and the employee role.

Language and Sender Nationality Guidelines:

    For internal emails, the sender's nationality/language should match the company's headquarter national language.
    For external emails received from clients who are from the same country as the company, also use the company's national language.
    For external emails from international clients, follow standard business norms by using English.

Employee_profile:{employee}
Company_profile:{company}

The emails should be received only.
For each email, only provide these elements:
email_description: A succinct summary outlining the email's purpose, key points, and context, all presented in the third person.
sender: The identity or role of the sender, including a generated name.
tone: A clear description of the email's tone to set the expected mood.
style: A clear description of the email's style, avoid precious formalisms.
length: A single word that characterizes the email's length.
receiver_info: Specifies whether the email is addressed to an individual or a larger group within the organization.
"""

scenario_ham_schema = {
    "name": "scenario_ham",
    "description": "Generate ham email scenarios for a given employee and company profile",
    "parameters": {
        "type": "object",
        "properties": {
            "email_scenarios": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "email_description": {
                            "type": "string",
                            "description": "A succinct summary outlining the email's purpose, key points, and context, all presented in the third person."
                        },
                        "sender": {
                            "type": "string",
                            "description": "The identity or role of the sender, including a generated name."
                        },
                        "tone": {
                            "type": "string",
                            "description": "A clear description of the email's tone and style to set the expected mood."
                        },
                        "style": {
                            "type": "string",
                            "description": "A clear description of the email's style, avoiding precious formalisms."
                        },
                        "length": {
                            "type": "string",
                            "description": "A single word that characterizes the email's length."
                        },
                        "receiver_info": {
                            "type": "string",
                            "description": "Specifies whether the email is addressed to an individual or a larger group within the organization."
                        }
                    },
                    "required": [
                        "email_description",
                        "sender",
                        "tone",
                        "style",
                        "length",
                        "receiver_info"
                    ],
                    "additionalProperties": False
                },
            }
        },
        "required": ["email_scenarios"],
        "additionalProperties": False
    }
}

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ],
    functions=[scenario_ham_schema],
    function_call={"name": "scenario_ham"},
    temperature=1.2
)

In [ ]:
ham_scenario = json.loads(response.choices[0].message.function_call.arguments)['email_scenarios'][0]

for key, value in ham_scenario.items():
    print(f"{key}: {value}")

email_description: Giulia receives an update from the marketing department regarding an upcoming promotional event for the 'Reserve Collection', emphasizing the need for her input on wine selection and production timelines.
sender: Marco Rossi, Marketing Manager
tone: Collaborative and proactive
style: Direct and straightforward
length: Short
receiver_info: Individual


## Ham Email creation

In [ ]:
system_prompt = """
You are an expert in writing professional work emails, adapting tone, style, and language to match workplace norms and recipient profiles.
"""

user_prompt = f"""
Generate only 1 email based on each set of email characteristics.
Create a plausible but realistic subject and body email that "Receiver_profile" employee might receive in his work inbox based on "Email characteristics".

Adaptive Language Use:
Use the language based on the sender Name in Email Characteristics.
In general:
  For internal emails, use the company’s national language (e.g., Italian for a company based in Italy).
  For external emails from clients coming from the same country of the company, use the company's national language (e.g., Italian for a company based in Italy).
  For external emails, follow standard business norms (e.g., English for international clients).
  Important: Subject and body must be written in the same language.
  If the subject is written in Italian then also the body must be written in Italian.
  If the subject is written in English then also the body must be written in English.


Realistic & Plausible Details:
Invent all elements (names, projects, deadlines, tools) to align with the company’s sector and the employee’s role.
Links: When including links, generate realistic URLs, also using random characters in the appropriate parts of it. Provide exclusively the link.

Contextual Tone, Length and Structure:
Match the tone to the sender/recipient dynamics.
Ensure the email feel human-written, with a natural flow and minor conversational quirks.
The opening, when inserted should sound natural, avoiding being precious and extremely formal.
Avoid the usage of standard and precious opening and introduction, be more creative or avoid their usage.
Avoid robotic or overly formal phrasing.

Alignment with Profiles:
Reflect the employee’s role, current projects, and technical proficiency.

The email details(content, sender, tone, style, length and who is the receiver) should be based on the following characteristics:

Email characteristics:{ham_scenario}
Receiver_profile:{employee}
Company_profile:{company}
Generate an email for each different "email characteristic".
"""



ham_email_schema = {
  "name": "email",
  "description": "Generate an email for a specific employee in a company context",
  "parameters": {
    "type": "object",
    "properties": {
      "identifier": {
        "type": "string",
        "description": "A unique identifier in the format 'CompanyName-EmployeeName'"
      },
      "emails": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "Subject": {
              "type": "string",
              "description": "A clear, specific subject line in the appropriate language"
            },
            "Body": {
              "type": "string",
              "description": "Contextually appropriate content in the same language of the subject"
            }
          },
          "required": ["Subject", "Body"],
          "additionalProperties": False
        }
      }
    },
    "required": ["identifier", "emails"],
    "additionalProperties": False
  }
}


response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ],
    functions=[ham_email_schema],
    function_call={"name": "email"},
    temperature=1
)



In [ ]:
ham_email = json.loads(response.choices[0].message.function_call.arguments)['emails'][0]

for key, value in ham_email.items():
    print(f"{key}: {value}")


Subject: Aggiornamento sull'evento promozionale 'Reserve Collection'
Body: Ciao Giulia,

Spero tu stia bene. Volevo informarti che stiamo pianificando un evento promozionale importante per la 'Reserve Collection' e abbiamo bisogno del tuo prezioso contributo per la selezione dei vini. 

Vorremmo poter contare sul tuo expertise per definire i dettagli delle tempistiche di produzione, in modo da allinearci con le nostre strategie di marketing. Sarebbe perfetto se potessimo rivedere insieme le opzioni questa settimana.

Fammi sapere quando avresti tempo per un incontro, così possiamo discutere più in dettaglio.

Grazie mille per il tuo aiuto!

Un caro saluto,

Marco Rossi
Marketing Manager
VinoVeritas S.p.A.


## Phishing Email Scenario Creation

In [ ]:
system_prompt = """
You are an AI expert in recognizing phishing email communication and in writing phishing email characteristics based on employee's profile and company details.
"""

user_prompt = f"""
Generate appropriate characteristics for a phishing email that the employee might receive.

Employee_profile: {employee}
Company_profile: {company}

Combination of phishing type and customization level:
- Credential harvesting - generic phishing


Generate the following characteristics for each of the previous combinations:
- phishing_type: (credential harvesting, malware installation, BEC with specified subtype)
- customization: (spear phishing or generic phishing)
- objective: The goal the attacker aims to achieve through the email.
- impersonated_identity: The entity or individual the attacker pretends to be in the phishing email.
- method: The specific mechanism used to deceive the recipient.
- social_engineering_technique: The psychological manipulation strategies employed to deceive the recipient into taking the desired action.
- tone_and_style: The writing style and emotional tone used in the phishing email to elicit a response.
- length: A single word that characterizes the email's length.
"""

scenario_phishing_schema = {
    "name": "scenario_phishing",
    "description": "Generate phishing email scenarios for a given employee and company profile",
    "parameters": {
        "type": "object",
        "properties": {
            "phishing_scenario": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "phishing_type": {
                            "type": "string",
                            "description": "(credential harvesting, malware installation, BEC)"
                        },
                        "customization": {
                            "type": "string",
                            "description": "(spear phishing or generic phishing)"
                        },
                        "objective": {
                            "type": "string",
                            "description": "The primary goal the attacker aims to achieve through the email."
                        },
                        "impersonated_identity": {
                            "type": "string",
                            "description": "The entity or individual the attacker pretends to be in the phishing email."
                        },
                        "method": {
                            "type": "string",
                            "description": "The specific mechanism used to deceive the recipient."
                        },
                        "social_engineering_technique": {
                            "type": "string",
                            "description": "The psychological manipulation strategies employed to deceive the recipient into taking the desired action."
                        },
                        "tone_and_style": {
                            "type": "string",
                            "description": "The writing style and emotional tone used in the email to elicit a response."
                        },
                        "length": {
                            "type": "string",
                            "description": "A single word that characterizes the email's length."
                        }
                    },
                    "required": [
                        "phishing_type",
                        "customization",
                        "objective",
                        "impersonated_identity",
                        "method",
                        "social_engineering_technique",
                        "tone_and_style",
                        "length"
                    ],
                    "additionalProperties": False
                }
            }
        },
        "required": ["phishing_scenario"],
        "additionalProperties": False
    }
}

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    functions=[scenario_phishing_schema],
    function_call={"name": "scenario_phishing"},
    temperature=1.2
)


In [ ]:
phishing_scenario = json.loads(response.choices[0].message.function_call.arguments)['phishing_scenario'][0]

for key, value in phishing_scenario.items():
    print(f"{key}: {value}")


phishing_type: credential harvesting
customization: generic phishing
objective: To obtain Giulia's login credentials for the company's internal systems.
impersonated_identity: VinoVeritas IT Department
method: A fake email appearing to be from the IT department requesting a password reset verification.
social_engineering_technique: Creating a sense of urgency by noting a supposed security breach that requires immediate action.
tone_and_style: Professional but urgent, designed to instill concern and prompt quick responses.
length: brief


## Phishing Email Creation

In [ ]:
system_prompt = """
You are an expert in writing emails given the email characteristics.
"""

user_prompt = f"""
Your task is to generate five distinct emails based on the provided "Email characteristics", tailoring each email to target the "Receiver_profile", who works at "Company_profile".

For each email, adhere to the following guidelines:

Email Generation Guidelines:

1.  Adaptive Language:
    Use language appropriate for the sender described in "impersonated_identity" within each "Email characteristic".
                   In general:
                          If "impersonated_identity" coming from country of the company, use the company's national language (e.g., Italian for a company based in Italy).
                          If "impersonated_identity" is an international company,person or service provider use English.
                          Important: Subject and body must be written in the same language.
                          If the subject is written in Italian then also the body must be written in Italian.
                          If the subject is written in English then also the body must be written in English.

2.  Realistic & Plausible Details (Crucial):
    Invent all necessary details to make the emails highly realistic (eg: names, projects, deadlines, tools, bank details, emails) without using placeholders.
    Every detail must be concrete and contribute to the email's believability.
    When you include links, represent them only as `<<link>>`. This should be plausible in the context of the email.

3.  Generic vs. Spear Phishing Customization:
    If the "customization" is specified as "generic phishing", ensure the email content, salutations, and overall personalization are generic. Avoid using details from the "Receiver_profile" or "Company_profile" directly. The email should appear as a generic phishing attempt.
    If the "customization" is "spear phishing", leverage the "Receiver_profile" and "Company_profile" to create highly targeted and believable emails.

4.  Content Based on "Email Characteristics":
    For each of the five "Email characteristics" sets provided below, generate a unique email that precisely matches the described:
        Email characteristics:{phishing_scenario}

Output Format:
    For each of the five "Email characteristic" sets, clearly output the generated email with a Subject and Body.
Data to use for email generation:


Receiver_profile:{employee}
Company_profile:{company}

Output Format:
For each "email characteristic", provide:
Subject: [A clear, specific subject line in the appropriate language]
Body: [Contextually appropriate content in the same language of the subject]
"""



ham_email_schema = {
  "name": "phishing_email",
  "description": "Generate an email for a specific employee in a company context",
  "parameters": {
    "type": "object",
    "properties": {
      "identifier": {
        "type": "string",
        "description": "A unique identifier in the format 'CompanyName-EmployeeName'"
      },
      "emails": {
        "type": "array",
        "items": {
          "type": "object",
          "properties": {
            "Subject": {
              "type": "string",
              "description": "A clear, specific subject line in the appropriate language"
            },
            "Body": {
              "type": "string",
              "description": "Contextually appropriate content in the same language of the subject"
            }
          },
          "required": ["Subject", "Body"],
          "additionalProperties": False
        }
      }
    },
    "required": ["identifier", "emails"],
    "additionalProperties": False
  }
}


response = client.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt}
    ],
    functions = [ham_email_schema],
    function_call = {"name": "phishing_email"},
    temperature = 1
)

In [ ]:
phishing_emails = json.loads(response.choices[0].message.function_call.arguments)['emails'][0]

for key, value in phishing_emails.items():
    print(f"{key}: {value}")

Subject: URGENTE: Verifica delle Credenziali Necessaria
Body: Gentile Giulia,

A causa di una recente violazione della sicurezza nei nostri sistemi interni, è necessario aggiornare immediatamente le vostre credenziali. Si prega di seguire il link sottostante per verificare il proprio account e resettare la password:

<<link>>

Si prega di completare questa operazione entro le prossime 24 ore per evitare blocchi dell'account.

Grazie per la vostra collaborazione.

Cordiali saluti,

Il Dipartimento IT di VinoVeritas
